### Предсказание будущих трат клиентов

#### Описание задачи

Одним из самых ценных источников информации о клиенте являются данные о банковских транзакциях. Необходимо предсказать будущие траты клиента, используя информацию о ранее совершенных тратах по последовательности МСС-кодов транзакций (вот [статья](https://www.banki.ru/wikibank/mcc-kod/) про МСС и [описание MCC-кодов](https://mcc-codes.ru/code)).

Метрики качества **MAP@10** и **MNDCG@10**

<div style="border:solid #64EDC1 2px; padding: 40px">

```MAP@K - Mean Average Precision at K```

$$\Large AP@K = \frac {\sum_{i=1}^{N} \frac {\sum_{j=1}^{K}\mathbb{1}_{r_{ij}}}{K} \times \mathbb{1}_{Rel_i}} {|Rel|}$$

```Average Precision at K (AP@K)``` вычисляет среднюю точность для предсказаний до позиции K

- $\mathbb{1}_{r_{ij}}$ - индикаторная функция, показывающая, яаляется ли элемент в позиции j положительным для пользователя i, т.е. с ним было взаимодействие (1 - было, 0 - не было)
- $\mathbb{1}_{Rel_i}$ - индикаторная функция, показывающая, есть ли у пользователя i вообще положительные элементы (1 - да, 0 - нет)
- |Rel| - общее количество положительных элементов для всех пользователей.
- N - количество пользователей в выборке.

$$\Large MAP@K(i) = \frac{1}{N} \sum_{n=1}^N AP@K(i)_n$$

```Mean Average Precision at K (MAP@K)``` вычисляет среднюю точность для всех пользователей до позиции K

- AP@K_i - Average Precision at K для пользователя i.
- N - количество пользователей в выборке.
    
</div>

<div style="border:solid #64EDC1 2px; padding: 40px">

```MNDCG@K - Normalized Discounted Cumulative Gain at K```

$$\Large DCG@K(i) = \sum_{j=1}^{K}\frac{\mathbb{1}_{r_{ij}}}{\log_2 (j+2)}$$

```Discounted Cumulative Gain at K (DCG@K)``` вычисляет сумму значений, пропорциональных релевантности, для предсказаний до позиции K

$\Large \mathbb{1}_{r_{ij}}$ -- индикаторная функция показывает что пользователь $i$ провзаимодействовал с продуктом $j$ (1 - да, 0 - нет)

$\log2(j+2)$ - логарифм по основанию 2 от j+2, используется для учета понижения значения с увеличением позиции. 2 - константа (можно уменьшить константу-шаг)

$$\Large IDCG@K(i) = max(DCG@K(i)) = \sum_{j=1}^{K}\frac{\mathbb{1}_{j\le|Rel_i|}}{\log_2 (j+2)}$$

```Ideal Discounted Cumulative Gain at K (IDCG@K)``` представляет собой максимально возможное значение DCG@K для пользователя i и рекомендаций длины K

$\Large |Rel_i|$ -- количество релевантных продуктов для пользователя $i$

$\mathbb{1}{j\leq|Reli|}$ - индикаторная функция, которая равна 1, если j $\leq$ количеству релевантных элементов $|Reli|$ для пользователя i, иначе равна 0.

$$\Large nDCG@K(i) = \frac {DCG@K(i)}{IDCG@K(i)}$$

```Normalized Discounted Cumulative Gain at K (nDCG@K)``` вычисляет относительное значение DCG@K по отношению к IDCG@K

$$\Large MnDCG@K(i) = \frac{1}{N} \sum_{n=1}^N nDCG@K(i)_n$$

```Mean Normalized Discounted Cumulative Gain at K (MnDCG@K)``` усредняет значения nDCG@K для всех пользователей
        
</div>

_____

Оценивать вероятность взаимодействия пользователя с элементом будем при помощи алгоритмв ALS (Alternative Least Squares, он минимизирует суммарную квадратичную ошибку между фактическими и предсказанными взаимодействиями

<div style="border:solid #64EDC1 2px; padding: 40px">

$$\Large \hat{r}{ui} = \alpha \cdot \textbf{x}u \cdot (\textbf{y}i)^T + \textbf{b}u + \textbf{c}i + \mu$$

- $\hat{r}{ui}$ - предсказанное взаимодействие пользователя u с элементом i,
- $\textbf{x}u$ - вектор-признак пользователя u,
- $\textbf{y}i$ - вектор-признак элемента i,
- $\alpha$ - коэффициент, отвечающий за влияние признаков пользователя и элемента на предсказание,
- $\textbf{b}u$ - смещение (bias) пользователя u,
- $\textbf{c}i$ - смещение (bias) элемента i,
- $\mu$ - средняя оценка или среднее взаимодействие в тренировочной выборке.

Алгоритм ALS выполняет итерации, чтобы оптимизировать векторы-признаки пользователей и элементов, а также смещения пользователей и элементов, чтобы минимизировать ошибку предсказания взаимодействий.
    
</div>

_____

импортируем библиотеки:

In [1]:
import joblib
import os
import pickle
import warnings
from tqdm import tqdm

from collections import Counter, OrderedDict
import numpy as np
import pandas as pd

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, PolynomialFeatures, RobustScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

from scipy.sparse import csr_matrix
from statistics import mode, median
import lightgbm as lgb

from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import bm25_weight, tfidf_weight

warnings.filterwarnings('ignore')

метрика - MAP@K:

In [2]:
# https://github.com/benhamner/Metrics/blob/master/Python/ml_metrics/average_precision.py
def apk(actual, predicted, k=10):
    if len(predicted) > k:
        predicted = predicted[:k]
    score = 0.0
    num_hits = 0.0
    for i, p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)
    if not actual:
        return 0.0

    return score / min(len(actual), k)

def mapk(actual, predicted, k=10):
    return np.mean([apk(a, p, k) for a, p in zip(actual, predicted)])

метрика - MNDCG@K:

In [3]:
def dcg_at_k(recommended_list, bought_list, k):
    """вычисляет DCG@k для заданных списков рекомендаций и фактических items"""
    
    dcg = 0.0
    for i in range(k):
        if recommended_list[i] in bought_list:
            dcg += 1.0 / np.log2(i + 2)  # числитель: считаем только совпадения, 2 - константа
    return dcg


def ndcg_at_k(recommended_list, bought_list, k):
    """расчет NDCG@k"""

    idcg = dcg_at_k(bought_list, bought_list, k)
    if idcg == 0:
        return 0.0
    dcg = dcg_at_k(recommended_list, bought_list, k)
    return dcg / idcg


def mean_ndcg_at_k(actual, predicted, k=10):
    """вычисляет среднее значение NDCG@k для набора actual и predicted"""
    
    ndcg_scores = []
    for a, p in zip(actual, predicted):
        ndcg_scores.append(ndcg_at_k(p, a, k))
    return np.mean(ndcg_scores)

загружаем файлы и подготавливаем их:

In [4]:
df_train = pd.read_csv('../../data/df_train.csv', sep=';')
df_test = pd.read_csv('../../data/df_test.csv', sep=';')
sample_forecast = pd.read_csv('../../data/submission_baseline_2.csv')
df_train.head(3)

,Id,Data,Target
0,0,"4814,4814,6010,6011,4814,6011,6011,4814,6011,6...","4814,4814,4814,4814,5411,4814,4814,4814,4814,4814"
1,1,"6011,6011,6011,6011,6011,6011,6011,4814,4814,4...","4814,6011,4814,6011,4814,4814,6011,4814,6011,4814"
2,2,"8021,6011,6011,6010,4829,4814,6011,6011,6011,6...","6011,6011,6010,4829,4829,6010,6011,6011,4814,6011"


In [5]:
def string_processing(df):
    """функция собирает значения наблюдений в списки"""
    
    df_ = df.copy()
    df_['Data'] = df_.Data.apply(lambda s: list(map(int, s.split(','))))
    try:
        df_['Target'] = df_.Target.apply(lambda s: list(map(int, s.split(','))))
    except AttributeError:
        pass
    return df_

In [6]:
df_train = string_processing(df_train)
df_test = string_processing(df_test)
display(df_train.head(3), df_test.head(3))

,Id,Data,Target
0,0,"[4814, 4814, 6010, 6011, 4814, 6011, 6011, 481...","[4814, 4814, 4814, 4814, 5411, 4814, 4814, 481..."
1,1,"[6011, 6011, 6011, 6011, 6011, 6011, 6011, 481...","[4814, 6011, 4814, 6011, 4814, 4814, 6011, 481..."
2,2,"[8021, 6011, 6011, 6010, 4829, 4814, 6011, 601...","[6011, 6011, 6010, 4829, 4829, 6010, 6011, 601..."


,Id,Data
0,0,"[4814, 4814, 6011, 6011, 6010, 6011, 6011, 481..."
1,1,"[6010, 6011, 6010, 5411, 5411, 5977, 6011, 601..."
2,2,"[4814, 6011, 5251, 6011, 7832, 5641, 5814, 482..."


In [7]:
print(f'размеры:\n{df_train.shape=}\n{df_test.shape=}')
if (df_train['Id'].values == df_test['Id'].values).all():
    print("Значения в столбце 'Id' идут в одинаковом порядке.")
else:
    print("Значения в столбце 'Id' не идут в одинаковом порядке.")

размеры:
df_train.shape=(7033, 3)
df_test.shape=(7033, 2)
Значения в столбце 'Id' идут в одинаковом порядке.


In [8]:
test_list = list(set([len(i) for i in df_test['Data']]))
train_list = list(set([len(i) for i in df_train['Data']]))
print(f' TRAIN\nминимум МСС:{min(train_list)}\nмаксимум МСС:{max(train_list)}\n',
      f'TEST\nминимум МСС:{min(test_list)}\nмаксимум МСС:{max(test_list)}')

 TRAIN
минимум МСС:40
максимум МСС:21101
 TEST
минимум МСС:40
максимум МСС:88771


проверяем, всё ли МСС правильного размера:

In [9]:
def get_codes(transactions):
    transactions_stats = sorted(
        Counter(transactions).items(), 
        key=lambda x: x[1], 
        reverse=True
    )
    return transactions_stats

df_transactions = df_train.copy()
df_transactions['transactions_code'] = df_transactions['Data'].apply(get_codes)
display(df_transactions.head(2))

MMS_codes, wrong_size, count_wrong_size = set(), set(), 0
for x in df_transactions['transactions_code'].values:
    for y in x:
        if len(str(y[0])) != 4:
            wrong_size.add(y[0])
            count_wrong_size += 1
        MMS_codes.add(y[0])
        
print(f' уникальный правильных кодов = {len(MMS_codes)}\n',
      f'ошибок в размере кода = {count_wrong_size}\n',
      f'это код: {wrong_size}')

,Id,Data,Target,transactions_code
0,0,"[4814, 4814, 6010, 6011, 4814, 6011, 6011, 481...","[4814, 4814, 4814, 4814, 5411, 4814, 4814, 481...","[(4814, 81), (6011, 75), (5311, 29), (5411, 28..."
1,1,"[6011, 6011, 6011, 6011, 6011, 6011, 6011, 481...","[4814, 6011, 4814, 6011, 4814, 4814, 6011, 481...","[(4814, 52), (6011, 41), (4829, 3), (6010, 2),..."


 уникальный правильных кодов = 184
 ошибок в размере кода = 76
 это код: {742}


всего кодов в трейне 184, один из них трёхзначный - это аетеринарка, ноль в начале потерялся

In [10]:
# расшифровка МСС-кодов из Data лежит здесь: 
with open('../../pickle/MSS_codes_dict.pickle', 'rb') as f:
    MMS_codes_dict_from_Data = pickle.load(f)
    
# расшифровка МСС-кодов вообще лежит здесь: 
with open('../../pickle/MSS_codes_dict.pickle', 'rb') as f:
    MMS_codes_dict = pickle.load(f)
MMS_codes_dict

{740: {'AA': 'Ветеринарные услуги'},
 760: {'AB': 'Сельскохозяйственные кооперативы'},
 780: {'AC': 'Услуги садоводства'},
 15: {'AD': 'Генеральные подрядчики – жилое и коммерческое строительство'},
 17: {'AE': 'Контрактные услуги'},
 27: {'AF': 'Издательства, печатное дело, набор текстов'},
 30: {'AG': 'Fly'},
 40: {'AH': 'ЖД-перевозка грузов'},
 41: {'AI': 'Пассажирские перевозки, в т.ч. скорая помощь'},
 42: {'AJ': 'Автотранстпортные перевозки, доставка и складское хранение'},
 43: {'AK': 'Неизвестный код, встречается у некоторых банков как категория Авиабилетов'},
 44: {'AL': 'Круизные линии, пристани и судна в лизинг'},
 45: {'AM': 'Авиалинии и аэропорты'},
 47: {'AN': 'Туроператоры, платные дороги и неклассифицированные пассажирские перевозки'},
 48: {'AO': 'Телекоммуникационные услуги, провайдеры, телеграф'},
 49: {'AP': 'ЖКУ'},
 50: {'AQ': 'Опт - офисная мебель и оборудование, стройматериалы, коммерческое оборудование'},
 51: {'AR': 'Опт - канцелярия, лекарства, галантерея, хим

это вот ТОП-10, пригодятся

In [11]:
top10_codes = list(df_train['Data'].explode().value_counts().head(10).index)
top10_codes

[6011, 6010, 4814, 5411, 4829, 5499, 5541, 5912, 5331, 5812]

развернём матрицу, добавим ей признаков разных

In [12]:
def find_code(code):
    """для добавления нового столбца-котегории на основе МСС-кодов"""
    
    if len(str(code)) != 4:
        code = int(code)
    else:
        code = int(str(code)[:2])    
    if code in MMS_codes_dict.keys():
        return list(MMS_codes_dict[code].keys())[0]
    return 'XX'

In [13]:
def make_features(df):
    """Создание новых признаков в виде словарей, где ключ - уникальное значение МСС-кода для конкретного Id"""

    df_ = df.copy()
    # количество повторений МСС-кодов
    df_['freq'] = df_['Data'].apply(lambda x: {i:x.count(i) for i in set(x)})
    # среднее значение встречаемости МСС-кода в списке МСС-кодов
    df_['mean'] = df_['Data'].apply(lambda x: {i:sum([1 for j in x if j==i])/len(x)+0.0001 for i in set(x)})
    # медиана встречаемости МСС-кода в списке МСС-кодов
    # если количество встречаемости МСС-кода = 1, то выбирается первое значение из списка
    df_['median'] = df_['Data'].apply(lambda x: {i:sorted([j for j in x if j==i])[len([j for j in x if j==i])//2] if len([j for j in x if j==i]) > 1 else x[0] for i in set(x)})
    # мода встречаемости МСС-кода в списке МСС-кодов
    df_['mode'] = df_['Data'].apply(lambda x: {i: mode(x) for i in set(x)})
    # сумма всех значений МСС-кодов для данного МСС-кода в списке МСС-кодов
    df_['sum'] = df_['Data'].apply(lambda x: {i: sum(x) for i in set(x)})
    # количество уникальных значений МСС-кодов для данного МСС-кода в списке МСС-кодов
    df_['unique_count'] = df_['Data'].apply(lambda x: {i: len(set(x)) for i in set(x)})
    # среднее значение всех значений МСС-кодов в списке МСС-кодов для данного МСС-кода, независимо от Id
    df_['all_mean'] = df_['Data'].apply(lambda x: {i: sum(x) / len(x) for i in set(x)})
    # медианное значение всех значений МСС-кодов в списке МСС-кодов для данного МСС-кода, независимо от Id
    df_['all_median'] = df_['Data'].apply(lambda x: {i: median(x) for i in set(x)})
    # категория кода, найденная с помощью функции find_code()
    df_['code_category'] = df_['Data'].apply(lambda x: {i: find_code(i) for i in set(x)})
    return df_


def add_table(df, df_train, col_name):
    """функция разворачивает словарь со значениями, созданными в make_features, и возвращает новый признак"""
    
    df_ = df.copy()
    new_column = []
    grouped = df_.groupby('Id')
    for _, group in grouped:
        id_value = group['Id'].values[0]
        data_values = group['Data'].values
        dict_values = df_train.loc[df_train['Id'] == id_value, col_name].values[0]
        for data_value in data_values:
            data_freq = dict_values.get(data_value)
            new_column.append(data_freq)
    df_[col_name] = new_column
    return df_


def create_full_dataframe(df, df_train, col_names):
    """
    функция разворачивает датафрейм вертикально вызывает функцию add_table
    и наполняет датафрейм новыми признаками. В тестовой выборке после этого может быть небольшое
    кол-во NaN-ов, их функция заполняет значениями предыдущей строчки и возвращает готовый датафрейм
    """
    
    df_unrolled = pd.DataFrame({'Id': np.repeat(df['Id'], df['Data'].str.len()),
                                'Data': np.concatenate(df['Data'].values)})    
    for col_name in col_names:
        df_unrolled = add_table(df_unrolled, df_train, col_name)
    # Заполняем пропущенные значения предыдущими значениями строки (у test пара таких встретится)
    df_unrolled.fillna(method='ffill', inplace=True)

    return df_unrolled

вот что получаем после *make_features()*

In [14]:
df_train = make_features(df_train)
df_train.head(2)

,Id,Data,Target,freq,mean,median,mode,sum,unique_count,all_mean,all_median,code_category
0,0,"[4814, 4814, 6010, 6011, 4814, 6011, 6011, 481...","[4814, 4814, 4814, 4814, 5411, 4814, 4814, 481...","{5921: 2, 5411: 28, 4814: 81, 6010: 1, 6011: 7...","{5921: 0.009359259259259258, 5411: 0.129729629...","{5921: 5921, 5411: 5411, 4814: 4814, 6010: 481...","{5921: 4814, 5411: 4814, 4814: 4814, 6010: 481...","{5921: 1164138, 5411: 1164138, 4814: 1164138, ...","{5921: 6, 5411: 6, 4814: 6, 6010: 6, 6011: 6, ...","{5921: 5389.527777777777, 5411: 5389.527777777...","{5921: 5311.0, 5411: 5311.0, 4814: 5311.0, 601...","{5921: 'AZ', 5411: 'AU', 4814: 'AO', 6010: 'BA..."
1,1,"[6011, 6011, 6011, 6011, 6011, 6011, 6011, 481...","[4814, 6011, 4814, 6011, 4814, 4814, 6011, 481...","{5732: 2, 5541: 1, 4814: 52, 6010: 2, 6011: 41...","{5732: 0.0199019801980198, 5541: 0.01000099009...","{5732: 5732, 5541: 6011, 4814: 4814, 6010: 601...","{5732: 4814, 5541: 4814, 4814: 4814, 6010: 481...","{5732: 540291, 5541: 540291, 4814: 540291, 601...","{5732: 6, 5541: 6, 4814: 6, 6010: 6, 6011: 6, ...","{5732: 5349.415841584158, 5541: 5349.415841584...","{5732: 4814, 5541: 4814, 4814: 4814, 6010: 481...","{5732: 'AX', 5541: 'AV', 4814: 'AO', 6010: 'BA..."


In [15]:
col_names = df_train.columns.drop(['Id', 'Data', 'Target'])
col_names

Index(['freq', 'mean', 'median', 'mode', 'sum', 'unique_count', 'all_mean',
       'all_median', 'code_category'],
      dtype='object')

и с помощью create_full_dataframe разворачиваем датафрейм вертикально, теперь одной строке-наблюдению соотвестветствует одна пара *Id-Data*:

In [16]:
full_train_df = create_full_dataframe(df_train, df_train, col_names)

print(f'{full_train_df.shape=}')
full_train_df.head(3)

full_train_df.shape=(3328880, 11)


,Id,Data,freq,mean,median,mode,sum,unique_count,all_mean,all_median,code_category
0,0,4814,81,0.37510,4814,4814,1164138,6,5389.527778,5311.0,AO
0,0,4814,81,0.37510,4814,4814,1164138,6,5389.527778,5311.0,AO
0,0,6010,1,0.00473,4814,4814,1164138,6,5389.527778,5311.0,BA


Формируем разреженную матрицу:

In [17]:
user_item_matrix = pd.pivot_table(full_train_df,
                                  index='Id', columns='Data', 
                                  values='freq',
                                  aggfunc='count', 
                                  fill_value=0)

user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit
# переведем в формат saprse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()
user_item_matrix.head(3)

Data,742,1711,1731,1799,2741,3000,3351,3501,4111,4112,...,8299,8398,8641,8699,8999,9211,9222,9311,9399,9402
Id,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0


Делаем словари, в которых будут лежать реальные соответствия item_id и номера столбца:

In [18]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

далее код для получения рекомендаций и цикл, в котором переберём аж 672 модели, из которых выберем лучшую по метрике *MAP@10*:

In [19]:
def get_sort_recommendations(user, model, sparse_user_item, N=100):
    """функция для получения рекомендаций"""
    
    recs = model.recommend(userid=userid_to_id[user], 
                           user_items=sparse_user_item[userid_to_id[user]],
                           N=N,
                           filter_already_liked_items=False, 
                           recalculate_user=True)
    mask = recs[1].argsort()[::-1]
    res = [id_to_itemid[rec] for rec in recs[0][mask]]
    
    return res


def fit_(factors=100, regularization=0.001, iterations=15, 
         user_item_matrix=user_item_matrix, tfidf=False, bm25=False):
    """функция формирует и возвращает модель"""
    
    model = AlternatingLeastSquares(factors=factors, 
                                    regularization=regularization,
                                    iterations=iterations, 
                                    calculate_training_loss=True, 
                                    num_threads=4,
                                    random_state=42)
    if tfidf:
        tfidf_user_item_matrix = tfidf_weight(user_item_matrix.T).T
        model.fit(tfidf_user_item_matrix)
    elif bm25:
        bm25_user_item_matrix = bm25_weight(user_item_matrix.T).T.tocsr()
        model.fit(csr_matrix(bm25_user_item_matrix).tocsr())
    else:
        model.fit(sparse_user_item)
    return model

In [20]:
# # пустая таблица
# result_table = pd.DataFrame({'name': [], 'MAP@10': [], 'MNDCG@10': []})

In [21]:
# for f in tqdm(range(50, 401, 50)):
#     for r in np.logspace(-3, 0, num=4):
#         for i in range(5, 36, 5):
#             for model_type in ['als', 'tfidf', 'bm25']:
#                 if model_type == 'als':
#                     model = fit_(factors=f, regularization=r, iterations=i)
#                     col_name = f'als, factors={f}, regularization={r}, iterations={i}' 
#                 elif model_type == 'tfidf':
#                     model = fit_(factors=f, regularization=r, iterations=i, tfidf=True)
#                     col_name = f'tfidf, factors={f}, regularization={r}, iterations={i}'
#                 elif model_type == 'bm25':
#                     model = fit_(factors=f, regularization=r, iterations=i, bm25=True)
#                     col_name = f'bm25, factors={f}, regularization={r}, iterations={i}'
                    
#                 df_train[col_name] =\
#                     df_train['Id'].apply(lambda x: get_sort_recommendations(x, model, sparse_user_item, N=10))

#                 result_table.loc[len(result_table)] =\
#                     [col_name, mapk(df_train['Target'], df_train[col_name]),
#                      mean_ndcg_at_k(df_train['Target'], df_train[col_name])]

сохраним таблицу в файл (уж больно долго обсчитывалась, больше 5-ти часов):

In [22]:
# result_table.to_csv('data/ALS/result_table.csv', index=False)
result_table = pd.read_csv('result_table.csv')

In [23]:
# сортировка результатов по MAP@10
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)

display(
    result_table.sort_values('MAP@10', ascending=False).head(),
    result_table.sort_values('MAP@10', ascending=False).tail()
)

,name,MAP@10,MNDCG@10
425,"bm25, factors=300, regularization=0.001, iterations=10",0.329608,0.502934
422,"bm25, factors=300, regularization=0.001, iterations=5",0.329466,0.502841
428,"bm25, factors=300, regularization=0.001, iterations=15",0.329280,0.502625
593,"bm25, factors=400, regularization=0.001, iterations=10",0.329116,0.502564
596,"bm25, factors=400, regularization=0.001, iterations=15",0.328734,0.502196


,name,MAP@10,MNDCG@10
6,"als, factors=50, regularization=0.001, iterations=15",0.175766,0.319515
9,"als, factors=50, regularization=0.001, iterations=20",0.174602,0.317062
18,"als, factors=50, regularization=0.001, iterations=35",0.174129,0.316141
12,"als, factors=50, regularization=0.001, iterations=25",0.173075,0.314988
15,"als, factors=50, regularization=0.001, iterations=30",0.172593,0.314165


это вот чтоб отдельно посчитать по лучшим параметрам

In [24]:
model = fit_(factors=300, regularization=0.001, iterations=10, bm25=True)
df_train['Predicted'] =\
    df_train['Id'].apply(lambda x: get_sort_recommendations(x, model, sparse_user_item, N=100))

print(f" MAP@10 = {mapk(df_train['Target'], df_train['Predicted'])}\n",
      f"MNDCG!10 = {mean_ndcg_at_k(df_train['Target'], df_train['Predicted'])}")

  0%|          | 0/10 [00:00<?, ?it/s]

 MAP@10 = 0.32960756998187685
 MNDCG!10 = 0.502936888021932


In [25]:
df_train.head(1)

,Id,Data,Target,freq,mean,median,mode,sum,unique_count,all_mean,all_median,code_category,Predicted
0,0,"[4814, 4814, 6010, 6011, 4814, 6011, 6011, 4814, 6011, 6011, 4814, 6011, 4814, 4814, 6011, 6011, 4814, 4814, 6011, 6011, 6011, 4814, 4814, 6011, 6011, 6011, 6011, 4814, 4814, 6011, 6011, 6011, 6011, 4814, 6011, 4814, 4814, 4814, 4814, 4814, 6011, 6011, 4814, 4814, 4814, 4814, 6011, 6011, 6011, 4814, 6011, 6011, 4814, 6011, 4814, 6011, 6011, 4814, 6011, 6011, 4814, 6011, 4814, 4814, 4814, 6011, 6011, 4814, 6011, 4814, 6011, 6011, 6011, 6011, 6011, 4814, 5411, 4814, 5311, 6011, 6011, 4814, 6011, 4814, 6011, 6011, 4814, 5311, 4814, 4814, 4814, 6011, 5411, 6011, 5311, 5411, 6011, 5411, 5411, 5311, ...]","[4814, 4814, 4814, 4814, 5411, 4814, 4814, 4814, 4814, 4814]","{5921: 2, 5411: 28, 4814: 81, 6010: 1, 6011: 75, 5311: 29}","{5921: 0.009359259259259258, 5411: 0.1297296296296296, 4814: 0.3751, 6010: 0.00472962962962963, 6011: 0.3473222222222222, 5311: 0.13435925925925926}","{5921: 5921, 5411: 5411, 4814: 4814, 6010: 4814, 6011: 6011, 5311: 5311}","{5921: 4814, 5411: 4814, 4814: 4814, 6010: 4814, 6011: 4814, 5311: 4814}","{5921: 1164138, 5411: 1164138, 4814: 1164138, 6010: 1164138, 6011: 1164138, 5311: 1164138}","{5921: 6, 5411: 6, 4814: 6, 6010: 6, 6011: 6, 5311: 6}","{5921: 5389.527777777777, 5411: 5389.527777777777, 4814: 5389.527777777777, 6010: 5389.527777777777, 6011: 5389.527777777777, 5311: 5389.527777777777}","{5921: 5311.0, 5411: 5311.0, 4814: 5311.0, 6010: 5311.0, 6011: 5311.0, 5311: 5311.0}","{5921: 'AZ', 5411: 'AU', 4814: 'AO', 6010: 'BA', 6011: 'BA', 5311: 'AT'}","[6011, 4814, 5311, 5411, 5921, 6010, 4829, 5499, 5912, 5331, 5541, 5977, 5691, 3000, 5999, 7995, 5948, 5719, 5814, 8999, 5944, 5732, 4816, 4511, 5712, 5945, 4900, 4112, 7922, 5812, 5942, 5735, 5641, 5631, 4121, 5995, 7994, 5261, 5655, 7011, 5661, 6051, 7298, 5714, 7523, 5992, 7230, 5211, 7991, 5192, 5967, 7311, 8043, 3501, 4789, 5611, 5949, 5621, 5811, 4111, 6211, 5968, 7210, 7221, 5441, 4215, 4784, 7999, 5309, 5947, 5722, 5651, 7993, 5462, 5994, 5039, 5976, 742, 7996, 5111, 8011, 5122, 5946, 7932, 4131, 5699, 7299, 7997, 4722, 5065, 5713, 5310, 2741, 5047, 5571, 7273, 7512, 7629, 8220, 5969]"


обучимся на лучшей модели и создадим столбец с предсказанием:

In [26]:
model = fit_(factors=300, regularization=0.001, iterations=10, bm25=True)
df_test['Predicted'] =\
    df_test['Id'].apply(lambda x: get_sort_recommendations(x, model, sparse_user_item, N=10))

  0%|          | 0/10 [00:00<?, ?it/s]

In [27]:
df_test.head(3)

,Id,Data,Predicted
0,0,"[4814, 4814, 6011, 6011, 6010, 6011, 6011, 4814, 6011, 4814, 6010, 6011, 6010, 4814, 6011, 6010, 5541, 5912, 4814, 6011, 6011, 6011, 6011, 4814, 4814, 6010, 5541, 6010, 6011, 4814, 6011, 6010, 4829, 4829, 5499, 4814, 6011, 6011, 4829, 6010, 4829, 6011, 4829, 6010, 6011, 6011, 6011, 6010, 4829, 6011, 4814, 4814, 6010, 6010, 6011, 4814, 6011, 5499, 6011, 4814, 6011, 6011, 4814, 4814, 6011, 6011, 6011, 6011, 6011, 6011, 6011, 6011, 6011, 4814, 6011, 4814, 5499, 6011, 4829, 6010, 6011, 4814, 4814, 5411, 6011, 6011, 5411, 5541, 5499, 6011, 6011, 5499, 6011, 4829, 5499, 6011, 6011, 4829, 4829, 5499, ...]","[6011, 4814, 5311, 5411, 5921, 6010, 4829, 5499, 5912, 5331]"
1,1,"[6010, 6011, 6010, 5411, 5411, 5977, 6011, 6010, 5411, 6011, 6011, 6010, 6011, 5533, 5661, 6011, 5411, 6011, 5691, 6011, 4814, 4814, 6011, 5983, 6011, 5331, 6011, 6011, 6010, 6011, 5411, 6011, 6010, 4812, 6011, 6010, 5533, 5451, 4814, 4814, 5411, 6011]","[6011, 4814, 4829, 6010, 5732, 5541, 5411, 5499, 5912, 5331]"
2,2,"[4814, 6011, 5251, 6011, 7832, 5641, 5814, 4829, 5311, 6010, 5411, 5311, 6011, 5411, 5722, 5499, 4814, 5814, 5499, 5661, 6011, 6011, 5411, 5814, 5499, 5999, 5499, 6011, 5411, 5964, 5964, 5411, 5411, 4814, 6011, 6011, 5499, 6011, 5814, 5499, 5499, 4814, 6011, 6010, 5499, 5814, 6011, 5411, 6011, 4814, 5814, 5499, 5814, 5499, 6011, 6011, 6010, 4814, 5661, 6010, 5499, 5814, 5411, 6011, 4814, 6010, 6010, 6011, 6011, 6012, 6011, 4814, 6010, 6011, 4814, 6010, 5814, 5499, 6011, 4814, 5499, 5814, 5912, 6010, 5814, 5814, 6011, 5812, 6011, 5812, 5814, 4814, 6010, 5499, 4829, 5499, 5499, 5814, 6011, 6010, ...]","[6010, 5814, 6011, 5411, 4829, 4814, 5331, 5999, 8021, 5912]"


сохраним файл с предсказанием:

In [28]:
# predicted_files = df_test[['Id', 'Predicted']]
# predicted_files.head(3)

In [29]:
# predicted_files['Predicted'] = predicted_files.Predicted.astype(str).str.replace(',', '')
# predicted_files.head(3)

In [30]:
# predicted_files.to_csv('data/ALS/10_submission_ALS_BM25.csv', index=False)

_____

_____

## вторая модель

хорошая попытка, но удалась плохо, не хватило мощности. Вторая модель ухудшает результат:

In [31]:
def create_dataframe(df_features, df_train):
    """
    функция создаёт датафрейм, в котором target - бинарный флаг, вероятность которого предсказываем
    """

    def reduce_mem_usage(df):
        """ функция снижения использования памяти"""

        start_mem = df.memory_usage().sum() / 1024**2
        print(f'Memory usage of dataframe is {start_mem:.2f} MB')

        for col in df.columns:
            col_type = df[col].dtype

            if col_type != object:
                c_min = df[col].min()
                c_max = df[col].max()
                if str(col_type)[:3] == 'int':
                    if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                        df[col] = df[col].astype(np.int8)
                    elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                        df[col] = df[col].astype(np.int16)
                    elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                        df[col] = df[col].astype(np.int32)
                    elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                        df[col] = df[col].astype(np.int64)
                else:
                    if c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                        df[col] = df[col].astype(np.float32)
                    else:
                        df[col] = df[col].astype(np.float64)
            else:
                df[col] = df[col].astype('category')

        end_mem = df.memory_usage().sum() / 1024**2
        print(f'использование памяти после оптимизации: {end_mem:.2f} MB')
        print(f'размер сократился на {(100 * (start_mem - end_mem) / start_mem):.2f} %')


    df_features = df_features.copy()
    df_train = df_train.copy()
    
    # столбцы из первого уровня, в т.ч. 100 предсказанных Data
    train_data_lvl_1 = df_train[['Id', 'Data', 'Predicted']]
    
    # создаём датафрейм для второго уровня
    users_lvl_2 = pd.DataFrame(df_features['Id'].unique())
    users_lvl_2.columns = ['Id']
    
    # получаем уникальные значения Id из train_data_lvl_1, чтобы ограничить users_lvl_2 
    # только теми Id, для которых у нас есть данные первого уровня
    
    train_users = train_data_lvl_1['Id'].unique()
    users_lvl_2 = users_lvl_2[users_lvl_2['Id'].isin(train_users)]
    
    # поиск предсказаний Predicted из train_data_lvl_1 для Id, входящих в users_lvl_2 
    # данные записываются в столбец candidates в users_lvl_2 
    # Если для какого-либо Id отсутствуют предсказания, такие строки удаляются.
    recs = train_data_lvl_1[train_data_lvl_1['Id'] \
                            .isin(users_lvl_2['Id'])][['Id', 'Predicted']]
    users_lvl_2 = users_lvl_2.merge(recs, on='Id', how='left')
    users_lvl_2.dropna(subset=['Predicted'], inplace=True) # на случай, если попались пропуски
    users_lvl_2.rename(columns={'Predicted': 'candidates'}, inplace=True)
    
    # каждая строка users_lvl_2 разбивается на отдельные строки, где каждая строка содержит 
    # только одно значение МСС-кода в столбце Data. Создается новый датафрейм, где 
    # столбец Data содержит эти отдельные значения МСС-кодов
    s = users_lvl_2.apply(lambda x: pd.Series(x['candidates']), axis=1).stack().reset_index(level=1, drop=True)
    s.name = 'Data'
    users_lvl_2 = users_lvl_2.drop('candidates', axis=1).join(s)
    
    # добавляем столбец flag со значением = 1 для каждой строки, 
    # он будет использоваться как метка для обучающих данных.
    users_lvl_2['flag'] = 1
    
    # создаём датафрейм targets, который содержит значения МСС-кодов из df_features и Id 
    # добавляеем в targets столбец 'target' = 1 для всех строк, которые 
    # совпадают с предсказаниями из users_lvl_2, и в 0 для остальных строк
    targets = df_features[['Id', 'Data']].copy()
    targets['target'] = 1
    targets = users_lvl_2.merge(targets, on=['Id', 'Data'], how='left')
    targets['target'].fillna(0, inplace= True)
    
    # удаляем столбец 'flag', он больше не нужен
    targets.drop('flag', axis=1, inplace=True)
    reduce_mem_usage(targets)
    reduce_mem_usage(df_features)
    
    # удаляем из df_features столбец Data, так как все значения МСС-кодов уже были обработаны
    df_features = df_features.drop(columns='Data')
    df_features = df_features.drop_duplicates()
    targets = targets.drop_duplicates()
    
    # targets объединяется с df_features по Id с right join. В итоговом 
    # датафрейме остаются только строки, которые присутствуют в df_features
    targets = targets.merge(df_features, on='Id', how='right')
    return targets

создаём датафрейм для второго уровня

#### ТРЕЙН:

In [32]:
df_lvl_2 = create_dataframe(full_train_df, df_train)
print(f'{df_lvl_2.shape=}')
df_lvl_2.head(3)

Memory usage of dataframe is 74.09 MB
использование памяти после оптимизации: 29.64 MB
размер сократился на 60.00 %
Memory usage of dataframe is 292.07 MB
использование памяти после оптимизации: 114.29 MB
размер сократился на 60.87 %
df_lvl_2.shape=(13588400, 12)


,Id,Data,target,freq,mean,median,mode,sum,unique_count,all_mean,all_median,code_category
0,0,6011,1.0,81,0.3751,4814,4814,1164138,6,5389.527832,5311.0,AO
1,0,4814,1.0,81,0.3751,4814,4814,1164138,6,5389.527832,5311.0,AO
2,0,5311,1.0,81,0.3751,4814,4814,1164138,6,5389.527832,5311.0,AO


#### ТЕСТ:

In [33]:
df_test = string_processing(pd.read_csv('../../data/df_test.csv', sep=';'))
df_test = make_features(df_test)
df_test.head(1)

,Id,Data,freq,mean,median,mode,sum,unique_count,all_mean,all_median,code_category
0,0,"[4814, 4814, 6011, 6011, 6010, 6011, 6011, 4814, 6011, 4814, 6010, 6011, 6010, 4814, 6011, 6010, 5541, 5912, 4814, 6011, 6011, 6011, 6011, 4814, 4814, 6010, 5541, 6010, 6011, 4814, 6011, 6010, 4829, 4829, 5499, 4814, 6011, 6011, 4829, 6010, 4829, 6011, 4829, 6010, 6011, 6011, 6011, 6010, 4829, 6011, 4814, 4814, 6010, 6010, 6011, 4814, 6011, 5499, 6011, 4814, 6011, 6011, 4814, 4814, 6011, 6011, 6011, 6011, 6011, 6011, 6011, 6011, 6011, 4814, 6011, 4814, 5499, 6011, 4829, 6010, 6011, 4814, 4814, 5411, 6011, 6011, 5411, 5541, 5499, 6011, 6011, 5499, 6011, 4829, 5499, 6011, 6011, 4829, 4829, 5499, ...]","{5411: 6, 5541: 8, 4814: 45, 5499: 14, 5399: 1, 5912: 1, 6010: 28, 6011: 96, 4829: 25}","{5411: 0.026885714285714284, 5541: 0.035814285714285715, 4814: 0.20099285714285714, 5499: 0.0626, 5399: 0.004564285714285714, 5912: 0.004564285714285714, 6010: 0.1251, 6011: 0.42867142857142854, 4829: 0.11170714285714287}","{5411: 5411, 5541: 5541, 4814: 4814, 5499: 5499, 5399: 4814, 5912: 4814, 6010: 6010, 6011: 6011, 4829: 4829}","{5411: 6011, 5541: 6011, 4814: 6011, 5499: 6011, 5399: 6011, 5912: 6011, 6010: 6011, 6011: 6011, 4829: 6011}","{5411: 1247782, 5541: 1247782, 4814: 1247782, 5499: 1247782, 5399: 1247782, 5912: 1247782, 6010: 1247782, 6011: 1247782, 4829: 1247782}","{5411: 9, 5541: 9, 4814: 9, 5499: 9, 5399: 9, 5912: 9, 6010: 9, 6011: 9, 4829: 9}","{5411: 5570.455357142857, 5541: 5570.455357142857, 4814: 5570.455357142857, 5499: 5570.455357142857, 5399: 5570.455357142857, 5912: 5570.455357142857, 6010: 5570.455357142857, 6011: 5570.455357142857, 4829: 5570.455357142857}","{5411: 6010.0, 5541: 6010.0, 4814: 6010.0, 5499: 6010.0, 5399: 6010.0, 5912: 6010.0, 6010: 6010.0, 6011: 6010.0, 4829: 6010.0}","{5411: 'AU', 5541: 'AV', 4814: 'AO', 5499: 'AU', 5399: 'AT', 5912: 'AZ', 6010: 'BA', 6011: 'BA', 4829: 'AO'}"


In [34]:
full_test_df = create_full_dataframe(df_test, df_train, col_names)
print(f'{full_test_df.shape=}')
full_test_df.head(3)

full_test_df.shape=(3353026, 11)


,Id,Data,freq,mean,median,mode,sum,unique_count,all_mean,all_median,code_category
0,0,4814,81.0,0.375100,4814.0,4814.0,1164138.0,6.0,5389.527778,5311.0,AO
0,0,4814,81.0,0.375100,4814.0,4814.0,1164138.0,6.0,5389.527778,5311.0,AO
0,0,6011,75.0,0.347322,6011.0,4814.0,1164138.0,6.0,5389.527778,5311.0,BA


ломовой размер, столько не вытянуть, оставим случайных 20% строк и размедим датасет на trai и test:

In [35]:
df_lvl_2 = df_lvl_2.sample(frac=0.2)
print(f'{df_lvl_2.shape=}')

df_lvl_2.shape=(2717680, 12)


а вот вторая модель - это бустинг **LGBMClassifier**:

<div style="border:solid #64EDC1 2px; padding: 40px">

$$\Large \hat{y}i = \text{argmax}k \left( \sum{j=1}^{M} fj(xi) \right)$$

- $\hat{y}i$ - предсказанная метка класса для i-го объекта,
- M - количество деревьев в модели,
- $fj(xi)$ - предсказание j-го дерева для i-го объекта.
    
</div>

In [36]:
columns_to_robust = ['freq', 'mean', 'median', 'mode', 'sum', 'unique_count', 'all_mean', 'all_median']
column_to_OHE = ['code_category']
columns_to_passthrough = ['Id', 'Data']
target = 'target'

In [37]:
y = df_lvl_2['target']
X = df_lvl_2.drop(columns='target')

In [38]:
model = lgb.LGBMClassifier(random_state=42, verbose=-1)

preprocessor = ColumnTransformer(transformers=[
    ('robust', RobustScaler(), columns_to_robust),
    ('label', OneHotEncoder(drop='first', handle_unknown='ignore'), column_to_OHE),
    ('pass', 'passthrough', columns_to_passthrough)
])

pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('poly', PolynomialFeatures(degree=2)),
    ('classifier', model)
])

params = {
    'classifier__n_estimators': [100, 290],
    'classifier__max_depth': [100, 200],
    'classifier__learning_rate': [0.1, 0.01]
}

grid = GridSearchCV(pipeline, param_grid=params, cv=3, scoring='roc_auc',
                    refit=True, n_jobs=3)

search = grid.fit(X, y)
print('---'*10, f'\nподобранные параметры:\n{search.best_params_}\n', '---'*10)

pipeline.set_params(**search.best_params_)
pipeline.fit(X, y)

сохраним обученный пайплайн:

In [39]:
folder_name = 'pipelines'
try:
    os.mkdir(folder_name)
except FileExistsError:
    pass

joblib.dump(pipeline, os.path.join(folder_name, f'pipeline_LGBMClassifier'))

загрузим обученный пайплайн:

In [40]:
# Укажите путь к файлу модели
model_file_path = os.path.join(folder_name, 'pipeline_LGBMClassifier')

# Загрузка модели из файла
loaded_model = joblib.load(model_file_path)
loaded_model

Получим прогнозы и ранжирование рекомендаций для каждого пользователя:

In [47]:
preds = loaded_model.predict_proba(full_test_df)[:, 1]  # вероятности класса "1"

In [48]:
full_test_df['score'] = preds
full_test_df['rank'] = full_test_df.groupby('Id')['score'].rank(ascending=False)

# Группируем по столбцу 'Id' и сортируем значения 'Data' по столбцу 'rank' в порядке убывания
df_sorted = full_test_df.groupby('Id').apply(lambda x: x.sort_values('rank', ascending=False)).reset_index(drop=True)
df_sorted

,Id,Data,freq,mean,median,mode,sum,unique_count,all_mean,all_median,code_category,score,rank
0,0,5399,75.0,0.347322,6011.0,4814.0,1164138.0,6.0,5389.527778,5311.0,BA,0.045599,224.0
1,0,5541,1.0,0.004730,4814.0,4814.0,1164138.0,6.0,5389.527778,5311.0,BA,0.074808,222.0
2,0,5541,1.0,0.004730,4814.0,4814.0,1164138.0,6.0,5389.527778,5311.0,BA,0.074808,222.0
3,0,5541,1.0,0.004730,4814.0,4814.0,1164138.0,6.0,5389.527778,5311.0,BA,0.074808,222.0
4,0,5541,75.0,0.347322,6011.0,4814.0,1164138.0,6.0,5389.527778,5311.0,BA,0.077831,218.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3353021,7032,6011,149.0,0.372600,6011.0,6011.0,2238023.0,26.0,5595.057500,5651.0,BA,0.994707,79.5
3353022,7032,6011,149.0,0.372600,6011.0,6011.0,2238023.0,26.0,5595.057500,5651.0,BA,0.994707,79.5
3353023,7032,6011,149.0,0.372600,6011.0,6011.0,2238023.0,26.0,5595.057500,5651.0,BA,0.994707,79.5
3353024,7032,6011,149.0,0.372600,6011.0,6011.0,2238023.0,26.0,5595.057500,5651.0,BA,0.994707,79.5


In [49]:
result_df = df_sorted.groupby('Id').agg({'Data': list, 'rank': list}).reset_index()
result_df.head(2)

,Id,Data,rank
0,0,"[5399, 5541, 5541, 5541, 5541, 5541, 5541, 5541, 5541, 5912, 5499, 5499, 5499, 5499, 5499, 5499, 5499, 5499, 5499, 5499, 5499, 5499, 5499, 5499, 4829, 4829, 4829, 4829, 4829, 4829, 4829, 4829, 4829, 4829, 4829, 4829, 4829, 4829, 4829, 4829, 4829, 4829, 4829, 4829, 4829, 4829, 4829, 4829, 4829, 5411, 5411, 5411, 5411, 5411, 5411, 6010, 6010, 6010, 6010, 6010, 6010, 6010, 6010, 6010, 6010, 6010, 6010, 6010, 6010, 6010, 6010, 6010, 6010, 6010, 6010, 6010, 6010, 6010, 6010, 6010, 6010, 6010, 6010, 4814, 4814, 4814, 4814, 4814, 4814, 4814, 4814, 4814, 4814, 4814, 4814, 4814, 4814, 4814, 4814, 4814, ...]","[224.0, 222.0, 222.0, 222.0, 218.5, 218.5, 218.5, 218.5, 216.0, 215.0, 213.0, 213.0, 213.0, 210.5, 210.5, 205.5, 205.5, 205.5, 205.5, 205.5, 205.5, 205.5, 205.5, 201.0, 197.0, 197.0, 197.0, 197.0, 197.0, 197.0, 197.0, 184.5, 184.5, 184.5, 184.5, 184.5, 184.5, 184.5, 184.5, 184.5, 184.5, 184.5, 184.5, 184.5, 184.5, 184.5, 184.5, 184.5, 184.5, 172.5, 172.5, 172.5, 172.5, 172.5, 172.5, 155.5, 155.5, 155.5, 155.5, 155.5, 155.5, 155.5, 155.5, 155.5, 155.5, 155.5, 155.5, 155.5, 155.5, 155.5, 155.5, 155.5, 155.5, 155.5, 155.5, 155.5, 155.5, 155.5, 155.5, 155.5, 155.5, 155.5, 155.5, 119.0, 119.0, 119.0, 119.0, 119.0, 119.0, 119.0, 119.0, 119.0, 119.0, 119.0, 119.0, 119.0, 119.0, 119.0, 119.0, 119.0, ...]"
1,1,"[5451, 5983, 5661, 5691, 5977, 5533, 5533, 5331, 4812, 5411, 5411, 5411, 5411, 5411, 5411, 6010, 6010, 6010, 6010, 6010, 6010, 6010, 4814, 4814, 4814, 4814, 6011, 6011, 6011, 6011, 6011, 6011, 6011, 6011, 6011, 6011, 6011, 6011, 6011, 6011, 6011, 6011]","[42.0, 41.0, 40.0, 39.0, 38.0, 37.0, 36.0, 35.0, 34.0, 33.0, 31.0, 31.0, 31.0, 28.5, 28.5, 24.0, 24.0, 24.0, 24.0, 24.0, 24.0, 24.0, 18.5, 18.5, 18.5, 18.5, 8.5, 8.5, 8.5, 8.5, 8.5, 8.5, 8.5, 8.5, 8.5, 8.5, 8.5, 8.5, 8.5, 8.5, 8.5, 8.5]"


и вот 10 первых в предсказании на отложенной выборке, смотрим, в файл сохраняем:

In [50]:
df_variant2 = result_df[['Id', 'Data']].copy()
df_variant2.rename(columns={'Data': 'Predicted'}, inplace=True)
df_variant2['Predicted'] = df_variant2['Predicted'] \
    .apply(lambda x: list(OrderedDict.fromkeys(x))) \
    .apply(lambda x: x+top10_codes if len(x) < 10 else x) \
    .apply(lambda x: x[:10])
df_variant2['Predicted'] = df_variant2.Predicted.astype(str).str.replace(',', '')
df_variant2

,Id,Predicted
0,0,[5399 5541 5912 5499 4829 5411 6010 4814 6011 6011]
1,1,[5451 5983 5661 5691 5977 5533 5331 4812 5411 6010]
2,2,[7299 4215 7311 5964 5251 6012 7832 5941 5311 4812]
3,3,[4121 3000 7011 4722 5309 5045 5964 5813 8999 5712]
4,4,[4814 6010 6011 6011 6010 4814 5411 4829 5499 5541]
...,...,...
7028,7028,[5976 8043 8099 5949 5948 5995 5621 5942 5943 5944]
7029,7029,[5331 5691 5912 5411 6011 6011 6010 4814 5411 4829]
7030,7030,[8011 5131 4722 8099 5655 5943 5651 5942 5661 5691]
7031,7031,[6300 5462 8099 5942 5699 5641 5732 5331 5533 5211]


In [51]:
df_variant2.to_csv('11_submission_ALS_BM25_LGBMClassifier.csv', index=False)

_____